**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. Notice that your `BigQueryHelper` object is called `chicago_taxi_helper`.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
chicago_taxi_helper = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                               dataset_name="chicago_taxi_trips")

Using Kaggle's public dataset BigQuery integration.


# Questions

This dataset contains a table with the information about taxi trips in the city of Chicago. You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

# 1) Find the Data
Before you can access the data, you need to find the table name with the data.

- Hint: Tab completion is helpful whenever you can't remember a command. Type `chicago_taxi_helper.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
# Your code here to find the table name
chicago_taxi_helper.list_tables()

['taxi_trips']

In [4]:
# write the table name as a string below
table_name = 'taxi_trips'

q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q_1.solution()

# 2) Peek at the data

Use `chicago_taxi_helper.head` to peak at the top few rows of the data. Inspect the data and see if the data quality is immediately obvious. After deciding whether you see any important issues, look at `q_2.solution()`.

In [7]:
# your code here
chicago_taxi_helper.head(table_name)

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,b1695bee74d250a2d105ec5fa662595b851d4a33,0ed0aa5bd95971653ff5ff7a2f89637f19f083644e5358...,2014-04-28 07:30:00+00:00,2014-04-28 07:30:00+00:00,0,0.0,None,None,None,None,725.0,0.0,0.0,0.0,725.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
1,b18cc4fc0e9691c6859d467dba90f506e746c65d,0c2af70fa4647cb1d6f0e8fcb08ad4359c5977513d2d69...,2015-07-13 06:15:00+00:00,2015-07-13 06:15:00+00:00,0,0.0,None,None,None,None,545.0,0.0,0.0,0.0,545.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
2,b195e5f2d975847c47985ad50dc9aa520c8625e1,74c65e298adb79ea78cdb8d051248b8c259995872a8907...,2014-03-15 08:30:00+00:00,2014-03-15 08:30:00+00:00,0,0.0,None,None,None,None,1325.0,0.0,0.0,0.0,1325.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
3,b19bfa2714907cd5029d3fe556413d71f8413cc9,c4b7d4a2434871020014e186ac8489b75620cc38c41f49...,2015-10-20 11:45:00+00:00,2015-10-20 11:45:00+00:00,0,0.0,None,None,None,None,605.0,0.0,0.0,0.0,605.0,Cash,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
4,b19c1c5d79808dd35f88af2068e12e169a536e32,735f3541fe218b30cab11d38c2d019c74efe94c117340a...,2013-05-28 07:45:00+00:00,2013-05-29 02:15:00+00:00,67140,0.0,None,None,None,None,2905.0,595.0,0.0,0.0,3500.0,Credit Card,Chicago Elite Cab Corp.,None,None,None,None,None,None


In [ ]:
q_2.solution()

# 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns
1. The year of the trips, which is called `year`
2. The number of trips in that year, called `num_trips`.

Hints:
- When using `GROUP BY` and `ORDER BY`, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The sql code to select the Year from `trip_start_timestamp` is <code>SELECT **EXTRACT(YEAR FROM trip_start_timestamp)**</code>.
- The FROM field can be a little tricky until you are used to it.  The format is
    1. A backick (the symbol \`)
    2. The project name. You can find this in the line where we created the BigQueryHelper object. In this case it is `bigquery-public-data`
    3. A period
    4. The dataset name. Also in the line where we created the BigQueryHelper object. In this case, it is `chicago_taxi_trips`
    5. A period
    6. The table name, which you already used as your answer in part 1.



In [25]:
rides_per_year_query = """ SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) num_trips 
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
GROUP BY year 
ORDER BY year
"""




rides_per_year_result = chicago_taxi_helper.query_to_pandas_safe(rides_per_year_query)

print(rides_per_year_result)
q_3.check()

   year  num_trips
0  2013   26870287
1  2014   31021726
2  2015   27400744
3  2016   19878276
4  2017    7689021


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [17]:
 q_3.hint()
 q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Start your query with  `SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) num_trips`

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

rides_per_year_query = """SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) num_trips
                    from `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    GROUP BY year
                    ORDER BY year"""

rides_per_year_result = chicago_taxi_helper.query_to_pandas_safe(rides_per_year_query)

```

# 4) Dive Slightly Deeper

It's odd that 2017 had so few rides. You should wonder whether it was systematic under-reporting throughout the year, or whether some months are missing.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways
1. Use a where clause to limit our query to data from 2017
2. Modify the extract statement to extract the month rather than the year


In [26]:

rides_per_month_query = """ SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, COUNT(1) num_trips 
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
GROUP BY month 
ORDER BY month
"""
rides_per_month_result = chicago_taxi_helper.query_to_pandas_safe(rides_per_month_query)

print(rides_per_month_result)
q_4.check()

   month  num_trips
0      1    1040262
1      2    1016484
2      3    1293745
3      4    1187501
4      5    1132690
5      6    1092557
6      7     925711
7      8         71


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q_4.hint()
#q_4.solution()

# 5) Write the Query

Let's step up the sophistication of your queries.

Write a query
- A column called `hour_of_day` and sort by this, which holds the result of extracting the hour from `trip_start_timestamp`.
- The count of total number of trips in each hour of the day (e.g. how many trips occurred between 6AM and 7AM, independent of which day it occurred on). Call this `num_trips`
- The average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. Call this column `avg_mph`. The value 3600 is used to convert from seconds to hours.

For 2017, we're missing August and everything after. So restrict your query to data meeting the following criteria
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0


You will use a CTE to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE. Instead you'll create those in the later `SELECT` statement).

This is a much harder query than anything you've written so far.  Good luck!


In [31]:
speeds_query = """
WITH RelevantRides AS
(SELECT trip_start_timestamp, trip_miles, trip_seconds
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_start_timestamp > '2017-01-01' and trip_start_timestamp <'2017-07-01' and trip_miles>0 and trip_seconds>0
)

SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, COUNT(1) num_trips, 3600*SUM(trip_miles)/SUM(trip_seconds) avg_mph
FROM RelevantRides
GROUP BY hour_of_day
ORDER BY hour_of_day
"""

# Set high max_gb_scanned because this query looks at more data
speeds_result = chicago_taxi_helper.query_to_pandas_safe(speeds_query, max_gb_scanned=20)

print(speeds_result)
q_5.check()

    hour_of_day  num_trips    avg_mph
0             1     427383  17.270590
1             2     398546  16.635014
2             3     379010  15.462462
3             4     388887  14.549414
4             5     421042  13.386499
5             6     461646  13.945169
6             7     510168  15.650150
7             8     556515  16.345441
8             9     553849  17.073965
9            10     507049  17.900230
10           11     475909  18.416820
11           12     453221  17.918006


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [29]:
 q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#cc5533">Sorry, no hints available for this question.</span>

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

speeds_query = """WITH RelevantRides AS
(SELECT EXTRACT(HOUR FROM trip_start_timestamp) hour_of_day, trip_miles, trip_seconds
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_start_timestamp > '2017-01-01' and trip_start_timestamp < '2017-07-01'
    AND trip_seconds > 0 AND trip_miles > 0
)

SELECT hour_of_day, count(1) num_trips, 3600 * SUM(trip_miles) / SUM(trip_seconds) avg_mph
FROM RelevantRides
GROUP BY hour_of_day
ORDER BY hour_of_day"""

speeds_result = chicago_taxi_helper.query_to_pandas_safe(speeds_query, max_gb_scanned=20)

```

That's a hard query. If you made good progress towards the solution, congratulations!

# 6. Ponder the Results
Something is wrong with either the raw data or our last query. What fact about the raw data doesn't seem right?

If you can identify the problem, how would you look at the raw data to verify that the problem is in the raw data and not just in your results? Check your answer below.

In [32]:
 q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
The results show rides with hours 1-12. But there should results in the afternoon (hours 13-24).

Perhaps the raw data has lost the distinction between AM and PM values.

You can review 200 rows of the raw data with the command `chicago_taxi_trips.head('taxi_trips', num_rows=200)`

You'll see that the timestamps are all in the AM hours (hours are less than or equal to 12.) 

At first you might worry that the data is coming back sorted by time, but the variety of dates suggests that's not the case. 
Part of data science is tracking down exactly this type of problem. If you were in an organization working on this, you could show the evidence you've just collected (e.g. the breakdown of trips by hour) to someone responsible for collecting the data, and help them debug the data collection and storage process using the results you've collected.


# Keep Going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in SQL course.


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

